# Importing Libraries

In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
import shutil
import os

# Similarity Based Features

This is the first part of designing graph features, at the first stage we will design similarit based features then we will design other type of features.

**Similarity Features** which mainly focus on the topological structure of the graph, are the most straightforward and oldest link prediction metrics. These methods try to figure the missing links out by assigning similarity score, s(vx;vy), between node pairs (vx and vy) using the structural property of the graphs.

Similarity-based methods consist of the three types:

1.   Local Similarity-Based Approaches.
2.   Global Similarity-Based Approaches.
3.   Quasi-Local Similarity-Based Approaches.


For Local Similarity-Based Approaches we will implement the following indices:

1.   Source Node Incoming
2.   Source Node Outcoming
3.   Destination Node Incoming
4.   Destination Node Outcoming
5.   Outcoming Intersection
6.   Incoming Intersection
1.   Common Neighbors
1.   Jaccard Index (JC)
2.   Salton Index (SL)
3.   Sørensen Index (SI)
4.   Preferential Attachment Index (PA)
5.   Adamic-Adar Index (AA)
7.   Hub Promoted Index (HP)
8.   Hub Depressed Index (HD)
9.   Leicht-Holme-Newman Index (LHN)
10.  Local Affinity Structure Index (LAS)
11.  CAR-Based Index (CAR)
12.  The Individual Attraction Index (IA)
13.  Functional Similarity Weight (FSW)





Common Neighbors is one of the most extensive information retrieval metrics for link prediction tasks, due to its high efficiency, despite
its simplicity. The idea behind CN is very intuitive; the probability of being linked for two nodes in the future is affected by the number of their common neighboring nodes, i.e., two nodes will highly probably establish a link if they have more shared nodes, It should be noted that the resulting score using CN is not normalized, and only shows the relative similarity of different node-pairs by considering shared nodes between them.

## Source / Destination Degree

In [ ]:
def get_degree(df):
  """
  Return incoming and outcoming degree of given dataframe
  This function would return two numbers the first one for outcoming degree and the second for incoming degree
  """
  out_degree, in_degree = [], []
  
  for i in df:
      try:
        out_degree.append(train_graph.out_degree(i))
      except:
        out_degree.append(0)
  
  for i in df:
      try:
        in_degree.append(train_graph.in_degree(i))
      except:
        in_degree.append(0)
  
  return out_degree, in_degree

## Intersection

In [ ]:
def get_intersection(source, destination):
  """
  Return incoming and outcoming intersection between source and destination nodes for a given dataframe
  This function would return two lists the first one for outcoming intersection and the second for incoming intersection
  """

  outcoming_intersection, incoming_intersection = [], []
  
  for node1, node2 in zip(source, destination):
    try:
      source_out = set(train_graph.successors(node1))
      destination_out = set(train_graph.successors(node2))
      outcoming_intersection.append(len(source_out.intersection(destination_out)))
    
    except:
      outcoming_intersection.append(0)

  for node1, node2 in zip(source, destination):
    try:
      source_in = set(train_graph.predecessors(node1))
      destination_in = set(train_graph.predecessors(node2))
      incoming_intersection.append(len(source_in.intersection(destination_in)))
    
    except:
      incoming_intersection.append(0)
  
  return outcoming_intersection, incoming_intersection

In [ ]:
# This implementation contains seperable code for each incoming and outcoming degree
# # AS common neigbhors is not implemented for directed graph
# # I will implement it using two function: common followers and common followee

# def common_followee(source, destination):
#   """
#   Define first part of common neigbhors index
#   """
  
#   try:
#     if len(list(train_graph.successors(source))) == 0 or len(list(train_graph.successors(source))) == 0:
#       return 0
  
#     source_followee = train_graph.successors(source)
#     destination_followee = train_graph.successors(destination)

#     return len(set(source_followee).intersection(set(destination_followee)))
  
#   except:
#     return 0

In [ ]:
# def common_followers(source, destination):
#   """
#   Define second part of common neigbhors index
#   """
#   try:
  
#     if len(list(train_graph.predecessors(source))) == 0 or len(list(train_graph.predecessors(source))) == 0:
#       return 0
  
#     source_followers = train_graph.predecessors(source)
#     destination_followers = train_graph.predecessors(destination)

#     return len(set(source_followers).intersection(set(destination_followers)))
  
#   except:
#     return 0

## Jaccard Index

Jaccard Index not only takes the number of common nodes into account as in CN, but it also normalizes it by considering the total set of numbers of shared and non-shared neighbors, hence it can be used to compair to result togeather, as it is define for neigbhoors in general for undirected graph I shall split this metric into two parts as with the most of the rest metrics.

### Jaccard For Outcoming

In [ ]:
def jaccard_for_outcoming(source, destination):
  
  """
  Calculate jaccard index for followee
  source      ====> source nodes
  destination ====> destination node
  """

  try: 
    if train_graph.out_degree(source) == 0 or train_graph.out_degree(destination) == 0:
      return 0

    intersection = set(train_graph.successors(source)).intersection(set(train_graph.successors(destination)))
    union = set(train_graph.successors(source)).union(set(train_graph.successors(destination)))
    
    return len(intersection) / len(union)
  
  except:
    return 0

### Jaccard For Incoming

In [ ]:
def jaccard_for_incoming(source, destination):
  
  """
  Calculate jaccard index for followers
  source      ====> source nodes
  destination ====> destination node
  """

  try:
    if train_graph.in_degree(source) == 0 or train_graph.in_degree(destination) == 0:
      return 0

    intersection = set(train_graph.predecessors(source)).intersection(set(train_graph.predecessors(destination)))
    union = set(train_graph.predecessors(source)).union(set(train_graph.predecessors(destination)))
    
    return len(intersection) / len(union)
  
  except:
    return 0

## Salton Index

Salton Index is also known as cosine similarity. It calculates the cosine angle between the two columns of the adjacency matrix and it is identified as the ratio of the number of shared neighbors of vx and vy to the square root of inner-product of their degrees.

### Salton For Outcoming

In [ ]:
def salton_for_outcoming(source, destination):
  """
  Calculate Salton Index for outcoming edges, this index also known as cosine distance and defined as:
  Number of common followee / square root of (number of followee of soruce times number of followee of destination)
  """
  try:
    source_out = train_graph.out_degree(source)
    destination_out = train_graph.out_degree(destination)
    if source_out == 0 or  destination_out == 0:
      return 0

    numerator = len(set(train_graph.successors(source)).intersection(set(train_graph.successors(destination))))
    denominator = (source_out * destination_out) ** 0.5
    
    return numerator / denominator
  
  except:
    return 0

### Salton For Incoming

In [ ]:
def salton_for_incoming(source, destination):
  
  """
  Calculate Salton Index for incoming edges, this index also known as cosine distance and defined as:
  Number of common followers / square root of (number of followers of soruce times number of followers of destination)
  """

  try:
    source_in = train_graph.in_degree(source)
    destination_in = train_graph.outin_degree(destination)
    if source_in == 0 or  destination_in == 0:
      return 0

    numerator = len(set(train_graph.predecessors(source)).intersection(set(train_graph.predecessors(destination))))
    denominator = (source_in * destination_in) ** 0.5
    
    return numerator / denominator
  
  except:
    return 0

## Sorensen Index

### Sorensen For Outcoming

In [ ]:
def sorensen_for_outcoming(source, destination):
  """
  Define sorensen index of nodes who source and destination nodes follow
  """
  try:
    source_out = train_graph.out_degree(source)
    destination_out = train_graph.out_degree(destination)
    if source_out == 0 or destination_out == 0:
      return 0

    numerator = len(set(train_graph.successors(source)).intersection(set(train_graph.successors(destination))))
    denominator = source_out + destination_out
    
    return numerator / denominator
  
  except:
    return 0

### Sorensen For Incoming

In [ ]:
def sorensen_for_incoming(source, destination):
  """
  Define sorensen index of nodes who follow source and destination nodes
  """
  try:
    source_in = train_graph.in_degree(source)
    destination_in = train_graph.in_degree(destination)
    if source_in == 0 or  destination_in == 0:
      return 0

    numerator = len(set(train_graph.predecessors(source)).intersection(set(train_graph.predecessors(destination))))
    denominator = source_out + destination_out
    
    return numerator / denominator
  
  except:
    return 0

New nodes joining the network are more likely to connect with the nodes with higher connections (hub) than the nodes with lower degrees, hence the result would be larger in this way.

## Preferential Attachment Index

### Preferential Attachment For Outcoming

In [ ]:
# This index is implementd in NetworkX for undirected graph only
# For that i will implement it using two parts for incoming and for outcoming

def outcoming_preferential_attachment(source, destination):
  """
  Implementing Preferential Attachmen Index of nodes who source and destination nodes follow
  """

  try:
    source_out = train_graph.out_degree(source)
    destination_out = train_graph.out_degree(destination)
    
    if source_out == 0 or destination_out == 0:
      return 0

    return source_out * destination_out
  except:
    return 0

### Preferential Attachment For Incoming

In [ ]:
def incoming_preferential_attachment(source, destination):
  """
  Implementing Preferential Attachmen Index of nodes who follow source and destination nodes
  """

  try:
    source_in = train_graph.in_degree(source)
    destination_in = train_graph.in_degree(destination)
    
    if source_in == 0 or destination_in == 0:
      return 0

    return source_in * destination_in
  except:
    return 0

## Adamic Adar Index

Lada Adamic and Eytan Adar index is defined by inverted sum of degrees of common neighbours for given two vertices as follow:
$$A(x,y)=\sum_{u \in N(x) \cap N(y)}\frac{1}{log(|N(u)|)}$$

We take intersection between two nodes, foreach node in interseaction u if it's neighbours large then it's chance to be a famous person or celebrity is high so no mean of x and y following each other (they follow the same famous person without knowing each other), if neighbours of u is small them there is high chance to x and y someone to follow the other or both.

**Note 1**: log is a monotonic function, in the formula we use it to reduce the impact so we don't divide by very big number of very small number.

**Note 2**: Adamic Adar index is defined for undirected graph, here we shall modifed it to fit with directed graph so we use it for same nodes followed by our two nodes soruce and destination nodes.

In [ ]:
def adamic_adar(source, destination):
  """
  Define Adamic Adar index for Directed Graph using only the nodes that source and destination nodes follow
  this implementation shall not consider nodes who follow source node and destination node
  """
  
  # define dictionary to hold number of nodes for a visited node
  # so we don't have to calculate it's neighbors many times
  map = dict()
  result = []
  
  for source_node, destination_node in zip(source, destination):
    try:
      # get the intersection between neighbors of source node and neighbors of destination node
      nodes = list(set(train_graph.successors(source_node)).intersection(set(train_graph.successors(destination_node))))
      
      if len(nodes) != 0:
        index = 0
        for node in nodes:
          
          if node in map:
            index += 1 / (map[node])
          
          else:
            neighbor_count = train_graph.in_degree(node)
            if neighbor_count == 0:
              map[node] = 0
            else:
              neighbors = np.log10(neighbor_count)
            map[node] = neighbors
            index += 1 / neighbors
        result.append(index)
      
      else:
        result.append(0)
    except:
      result.append(0)
  
  return result

## Hup Promoted

HP is determined by the ratio of the number of common neighbors of both source and destination to the minimum of degrees of source and destination. Here, link formation between lower degree nodes and the hubs is more promoted, while the formation of the connection between hub nodes are demoted.

### Hup Prompted For Outcoming

In [ ]:
def outcoming_hup_promoted(source, destination):

  """
  Define Hup Promoted Index for outcoming edges or for nodes that source and destination follow
  it defined as number of common successors over min number of successors between two nodes
  """

  result = []
  for source_node, destination_node in zip(source, destination):

    try:
      
      source_successors_len = train_graph.out_degree(source_node)
      destination_successros_len = train_graph.out_degree(destination_node)

      if source_successors_len == 0 or destination_successros_len == 0:
        result.append(0)
        continue
  
      numerator = len(set(train_graph.successors(source_node)).intersection(set(train_graph.successors(destination_node))))
      denominator = min(source_successors_len, destination_successros_len)

      result.append(numerator / denominator)
  
    except:
      result.append(0)
  
  return result

### Hup Prompted For Incoming

In [ ]:
def incoming_hup_promoted(source, destination):

  """
  Define Hup Promoted Index for incoming edges or for nodes that follow source and destination
  it defined as number of common predecessors over min number of predecessors between two nodes
  """

  result = []
  for source_node, destination_node in zip(source, destination):
    try:

      source_predecessors_len = train_graph.in_degree(source_node)
      destination_predecessors_len = train_graph.in_degree(destination_node)

      if source_predecessors_len == 0 or destination_predecessors_len == 0:
        result.append(0)
        continue
  
      numerator = len(set(train_graph.predecessors(source_node)).intersection(set(train_graph.predecessors(destination_node))))
      denominator = min(source_predecessors_len, destination_predecessors_len)

      result.append(numerator / denominator)

    except:
      result.append(0)

  return result

## Hup Depressed

The totally opposite analogy of HP is also considered and it is determined by the ratio of the number of common neighbors of both source and destination to the maximum of degrees of source and destination. Here, the link formation between lower degree nodes and link formation between hubs is promoted. However, the connection between hub nodes and lower degree nodes are demoted.

### Hup Depressed For Outcoming

In [ ]:
def outcoming_hup_depressed(source, destination):
  """
  Define Hup Depressed Index for outcoming edges or for nodes that source and destination follow
  it defined as number of common successors over max number of successors between two nodes
  """

  result = []
  for source_node, destination_node in zip(source, destination):

    try:
      
      source_successors_len = train_graph.out_degree(source_node)
      destination_successros_len = train_graph.out_degree(destination_node)

      if source_successors_len == 0 or destination_successros_len == 0:
        result.append(0)
        continue
  
      numerator = len(set(train_graph.successors(source_node)).intersection(set(train_graph.successors(destination_node))))
      denominator = max(source_successors_len, destination_successros_len)

      result.append(numerator / denominator)
  
    except:
      result.append(0)
  
  return result

### Hup Depressed For Incoming

In [ ]:
def incoming_hup_depressed(source, destination):

  """
  Define Hup Promoted Index for incoming edges or for nodes that follow source and destination
  it defined as number of common predecessors over max number of predecessors between two nodes
  """

  result = []
  for source_node, destination_node in zip(source, destination):
    try:

      source_predecessors_len = train_graph.in_degree(source_node)
      destination_predecessors_len = train_graph.in_degree(destination_node)

      if source_predecessors_len == 0 or destination_predecessors_len == 0:
        result.append(0)
        continue
  
      numerator = len(set(train_graph.predecessors(source_node)).intersection(set(train_graph.predecessors(destination_node))))
      denominator = max(source_predecessors_len, destination_predecessors_len)

      result.append(numerator / denominator)

    except:
      result.append(0)

  return result

## Leicht Holme Newman Index

### Leicht Holme Newman For Outcoming

In [ ]:
def outcoming_leicht(source, destination):
  """
  Define Leicht Holme Newman Index of outcoming edges
  it defined as number of intersection nodes over multiplication of length of each node from our two node's followee
  """

  result = []
  for source_node, destination_node in zip(source, destination):
    try:
      source_successors = train_graph.out_degree(source_node)
      destination_successors = train_graph.out_degree(destination_node)
      
      if source_successors == 0 or destination_successors == 0:
          result.append(0)
          continue
        
      numerator = len(set(train_graph.successors(source_node)).intersection(set(train_graph.successors(destination_node))))
      denominator = source_successors * destination_successors

      result.append(numerator / denominator)
    except:
      result.append(0)

  return result

### Leicht Holme Newman For Incoming

In [ ]:
def incoming_leicht(source, destination):
  """
  Define Leicht Holme Newman Index of incoming edges
  it defined as number of intersection nodes over multiplication of length of each node from our two node's followers
  """

  result = []
  for source_node, destination_node in zip(source, destination):
    
    try:
      source_predecessors = train_graph.in_degree(source_node)
      destination_predecessors = train_graph.in_degree(destination_node)
      
      if source_predecessors == 0 or destination_predecessors == 0:
          result.append(0)
          continue
        
      numerator = len(set(train_graph.predecessors(source_node)).intersection(set(train_graph.predecessors(destination_node))))
      denominator = source_predecessors * destination_predecessors

      result.append(numerator / denominator)

    except:
      result.append(0)

  return result

## Local Affinity Structure Index

Local Affinity Structure shows the affinity relationship between a pair of nodes and their common neighbors. The hypothesis is that a higher affinity of two nodes and their common neighbors increases the probability of getting connected.

### Local Affinity Structure For Outcoming

In [ ]:
def outcoming_local_affinity(source, destination):
  """
  Define Local Affinity Structure Index for outcoming edges of our two nodes
  This define as summing of two parts:
  Part1 ========> Intersection of two node's successors over number of first node successors
  Part2 ========> Intersection of two node's successors over number of second node successors
  """

  result = []
  for source_node, destination_node in zip(source, destination):
    try:
      source_successors = train_graph.out_degree(source_node)
      destination_successors = train_graph.out_degree(destination_node)
      
      if source_successors == 0 or destination_successors == 0:
          result.append(0)
          continue
        
      numerator = len(set(train_graph.successors(source_node)).intersection(set(train_graph.successors(destination_node))))
      metric = (numerator / source_successors) + (numerator / destination_successors)
      result.append(metric)
    except:
      result.append(0)

  return result

### Local Affinity Structure For Incoming

In [ ]:
def incoming_local_affinity(source, destination):
  """
  Define Local Affinity Structure Index for incoming edges of our two nodes
  This define as summing of two parts:
  Part1 ========> Intersection of two node's predecessors over number of first node predecessors
  Part2 ========> Intersection of two node's predecessors over number of second node predecessors
  """

  result = []
  for source_node, destination_node in zip(source, destination):
    try:
      source_predecessors = train_graph.in_degree(source_node)
      destination_predecessors = train_graph.in_degree(destination_node)
      
      if source_predecessors == 0 or destination_predecessors == 0:
          result.append(0)
          continue
        
      numerator = len(set(train_graph.successors(source_node)).intersection(set(train_graph.successors(destination_node))))
      metric = (numerator / source_predecessors) + (numerator / destination_predecessors)
      result.append(metric)
    except:
      result.append(0)

  return result

## Car Based Index

Car Based Index the consider second-level neighborhood which carries essential information regarding the topology of the network. Therefore, CAR filters these noises and considers nodes that are interlinked with neighbors mostly.

In [ ]:
def car_based_index(source, destination):
  """
  Car-Based Index of source and destination nodes is similar to Adamic Adar as it defined as follow:
  number of intersection nodes between our two nodes time number of incoming edges for each node in the intersection 
  """
  
  result = []
  for source_node, destination_node in zip(source, destination):
    index = 0
    
    try:
      nodes = list(set(train_graph.successors(source_node)).intersection(set(train_graph.successors(destination_node))))
      
      if len(nodes) != 0:
        for node in nodes:
          index += train_graph.in_degree(node) / 2
        
        result.append(len(nodes) * index)

      else:
        result.append(0)
    
    except:
      result.append(0)
  
  return result

## Individual Attraction Index

In [ ]:
def individual_attraction(source, destination):
  """
  Individual Attraction Index for outcoming edges is defined as follow:
  Intersection between source and destination + 2 over the intersection times number of predecessors of each node in the neighbhoor
  of source and destination togeather
  """

  result = []
  for source_node, destination_node in zip(source, destination):
    index = 0
    
    try:
      nodes = list(set(train_graph.successors(source_node)).intersection(set(train_graph.successors(destination_node))))
      if len(nodes) != 0:
        
        for node in nodes:
          index += (len(nodes) + 2) / ((len(nodes) * train_graph.in_degree(node)))
        result.append(index)
      
      else:
        result.append(0)
      
    except:
      result.append(0)
  
  return result

This index is first used in order to understand the similarity of physical or biochemical characteristics of proteins. Their motivation is based on the Czekanowski–Dice distance that is used in order to estimate the functional similarity of proteins.

## Functional Similarity Weight

### Functional Similarity Weight For Outcoming

In [ ]:
def out_func_sim_weight(source, destination, average_degree):
  """
  Functional Similarity Weight of outcoming edges which is defined as follow:
  2 * number of intersection between source and destination 
  over number of diffrence between neighbors of source and neighbors of destination
  plus two times number of intersection plus Beta
  Beta is define as follow:
  max(0, average neighbors of netowrk - number of diffrence + number of intersection )
  """

  result = []
  for source_node, destination_node in zip(source, destination):
    
    try:
      if train_graph.successors(source_node) == 0 or train_graph.successors(destination_node) == 0:
        result.append(0)
        continue
      
      s1, s2 = set(train_graph.successors(source_node)), set(train_graph.successors(destination_node))
      
      numerator = len(s1.intersection(s2))
      denominator = s1 - s2
      beta = max(0, average_degree - len(denominator) + numerator)

      result.append((((2 * numerator) / (len(denominator) + beta)) ** 2))
        
    except:
      result.append(0)
    
  return result

### Functional Similarity Weight For Incoming

In [ ]:
def in_func_sim_weight(source, destination, average_degree):
  """
  Functional Similarity Weight of incoming edges which is defined as follow:
  2 * number of intersection between source and destination 
  over number of diffrence between neighbors of source and neighbors of destination
  plus two times number of intersection plus Beta
  Beta is define as follow:
  max(0, average neighbors of netowrk - number of diffrence + number of intersection )
  """

  result = []
  for source_node, destination_node in zip(source, destination):
    
    try:
      if train_graph.predecessors(source_node) == 0 or train_graph.predecessors(destination_node) == 0:
        result.append(0)
        continue
      
      s1, s2 = set(train_graph.predecessors(source_node)), set(train_graph.predecessors(destination_node))
      
      numerator = len(s1.intersection(s2))
      denominator = s1 - s2
      beta = max(0, average_degree - len(denominator) + numerator)

      result.append((((2 * numerator) / (len(denominator) + beta)) ** 2))
        
    except:
      result.append(0)
    
  return result

**Refrence**: Ece C. Mutlu et al, 2020, REVIEW ON LEARNING AND EXTRACTING GRAPH FEATURES FOR LINK PREDICTION.